```

Chat Engine - Condense Plus Context Mode

This is a multi-step chat mode built on top of a retriever over your data.

For each chat interaction:

    First condense a conversation and latest user message to a standalone question
    Then build a context for the standalone question from a retriever,
    Then pass the context along with prompt and user message to LLM to generate a response.

This approach is simple, and works for questions directly related to the knowledge base and general interactions.
```

In [1]:
import os

# creating embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# reloading index

from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="embed_dir/")

# load index
index = load_index_from_storage(storage_context)

from llama_index.llms.ollama import Ollama

# ollama
Settings.llm = Ollama(model="llama2", request_timeout=180.0)

c:\Users\snehi\Desktop\Learning\.dev_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=10000)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=Settings.llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about discussions around coffee in r/indiacoffee subreddit"
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=False,
)

In [3]:
response = chat_engine.chat("do people prefer arabica or robusta coffee beans?")
print(response)

Hello there! *adjusts aviator glasses* It's a great question you ask. From what I've observed in the r/indiacoffee subreddit, it seems that many coffee enthusiasts tend to prefer Arabica over Robusta. The reason being is that Arabica beans are generally considered to have a more complex and nuanced flavor profile compared to Robusta.

Arabica beans contain more antioxidants and have a smoother, less bitter taste, which many people find more appealing. Additionally, Arabica beans are grown at high elevations and require more care and attention during the cultivation process, which can result in a higher quality final product.

On the other hand, Robusta beans are often seen as more bitter and have a harsher flavor profile. While they may have a harder punch of caffeine, many people find this trade-off to be not worth it. However, some specialty roasters do offer high-quality Robusta beans that are well-balanced and have a unique flavor profile that sets them apart from other types of co